In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
import subprocess
%matplotlib inline  

def startSimulation():
    # start current simulation
    subprocess.call(["xdotool", "mousemove", "596", "359"])
    subprocess.call(["xdotool", "click", "1"])
    subprocess.call(["xdotool", "click", "1"])
    subprocess.call(["xdotool", "click", "1"])
    subprocess.call(["xdotool", "mousemove", "369", "445"])
    subprocess.call(["xdotool", "click", "1"])

def exitSimulation():
    # exit current simulation
    subprocess.call(["xdotool", "mousemove", "389", "41"])
    subprocess.call(["xdotool", "click", "1"])
    subprocess.call(["xdotool", "key", "Escape"])
    
def startPIDcontroller(params, N):
    process = subprocess.Popen(["./build/pid", str(params[0]), str(params[1]), str(params[2]), str(N)], stdout=subprocess.PIPE)
    out, err = process.communicate()
    return out

# NOTE: We use params instead of tau_p, tau_d, tau_i
def run(params, n=1000):
    startSimulation()
    err = startPIDcontroller(params, n)
    writeToFile("./twiddle.out", params, err)
    exitSimulation()
    return err

# write each simulator run to a file
def writeToFile(fn, p, best_err):
    f = open(fn, "a")
    f.write(str(p) + ": " + str(best_err) + "\n")
    f.close()

# Make this tolerance bigger if you are timing out!
def twiddle(tol=0.2): 
    # Don't forget to call `make_robot` before every call of `run`!
    p = [0, 0, 0]
    dp = [1, 1, 1]
    best_err = run(p)
    
    # TODO: twiddle loop here
    while sum(dp) > tol:
        for i in range(len(dp)):
            p[i] += dp[i]
            err = run(p)
            if err < best_err:
                best_err = err
                dp[i] *= 1.1
            else:
                p[i] -= 2*dp[i]
                err = run(p)
                if err < best_err:
                    best_err = err
                    dp[i] *= 1.1
                else:
                    p[i] += dp[i];
                    dp[i]*=0.9
    
    return p, best_err


params, err = twiddle()
print("Final twiddle error = {}".format(err))


In [52]:
startSimulation()
err = startPIDcontroller([0.02,0,1.5], 200)
exitSimulation()
print(float(err))

323.609
